In [ ]:
#Name : Komal Dodiya
#Student ID : U01948955
#Subject : Data Science


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
year_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_y_tot_V2.0.csv', sep=';')

In [ ]:
year_df

In [ ]:
year_df.info()

In [ ]:
year_df.describe()


In [ ]:
time_cols = ['year']
time_col = None

for col in time_cols:
    if col in year_df.columns:
        time_col = col
        break
if time_col is not None:
  print(f'Time column identified as: {time_col}')
  # Further processing based on time_col, e.g.
  # Convert the time column to datetime objects if not already
  if not pd.api.types.is_datetime64_any_dtype(year_df[time_col]):
      try:
          year_df[time_col] = pd.to_datetime(year_df[time_col])
      except ValueError:
          print("Could not convert to datetime automatically. Please check the format of the time column.")
else:
    print("Time column not identified. Please check the dataframe columns.")

In [ ]:
from prophet import Prophet

# Prepare the data for Prophet
year_df = year_df.rename(columns={'year': 'ds', 'total_sunspots': 'y'})

# Create and fit the Prophet model
model = Prophet()
model.fit(year_df)

# Create future dates for prediction
future = model.make_future_dataframe(periods=365, freq='D')

# Make predictions
forecast = model.predict(future)

# Extract predictions for specific future dates
predictions_100 = forecast.iloc[-100]['yhat']
predictions_200 = forecast.iloc[-200]['yhat']
predictions_365 = forecast.iloc[-365]['yhat']

print(f"Sunspot prediction 100 days into the future: {predictions_100}")
print(f"Sunspot prediction 200 days into the future: {predictions_200}")
print(f"Sunspot prediction 365 days into the future: {predictions_365}")

fig1 = model.plot(forecast)
plt.show()

# Plot the components of the forecast
fig2 = model.plot_components(forecast)
plt.show()

In [ ]:
future_months = model.make_future_dataframe(periods=9, freq='MS') #MS: Month Start

# Make predictions
forecast_months = model.predict(future_months)

# Extract predictions for 1, 6, and 9 months into the future
predictions_1_month = forecast_months.iloc[-9]['yhat'] #last 9 months
predictions_6_months = forecast_months.iloc[-4]['yhat']
predictions_9_months = forecast_months.iloc[-1]['yhat']

print(f"Sunspot prediction 1 month into the future: {predictions_1_month}")
print(f"Sunspot prediction 6 months into the future: {predictions_6_months}")
print(f"Sunspot prediction 9 months into the future: {predictions_9_months}")

# Plotting
fig, ax = plt.subplots(figsize=(9,6))

#Plot the historical data
ax.plot(year_df['ds'], year_df['y'], label='Observed')

# Plot the forecast
ax.plot(forecast_months['ds'], forecast_months['yhat'], label='Forecast')

#Highlight predictions for 1,6 and 9 months
ax.scatter(forecast_months['ds'].iloc[-9], predictions_1_month, color='red', label='1 Month Prediction', s=50)
ax.scatter(forecast_months['ds'].iloc[-4], predictions_6_months, color='green', label='6 Months Prediction', s=50)
ax.scatter(forecast_months['ds'].iloc[-1], predictions_9_months, color='blue', label='9 Months Prediction', s=50)

ax.set_xlabel('Date')
ax.set_ylabel('Sunspots')
ax.set_title('Sunspot Predictions')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


fig2 = model.plot_components(forecast_months)
plt.show()

In [ ]:
# Predict sunspots for 1, 10, and 20 years into the future
future_years = model.make_future_dataframe(periods=20, freq='Y')
forecast_years = model.predict(future_years)

# Extract predictions
predictions_1_year = forecast_years.iloc[-20]['yhat']
predictions_10_year = forecast_years.iloc[-10]['yhat']
predictions_20_year = forecast_years.iloc[-1]['yhat']

print(f"Sunspot prediction 1 year into the future: {predictions_1_year}")
print(f"Sunspot prediction 10 years into the future: {predictions_10_year}")
print(f"Sunspot prediction 20 years into the future: {predictions_20_year}")

# Plotting
fig, ax = plt.subplots(figsize=(9, 6))

# Plot historical data
ax.plot(year_df['ds'], year_df['y'], label='Observed')

# Plot forecast
ax.plot(forecast_years['ds'], forecast_years['yhat'], label='Forecast')

# Highlight predictions
ax.scatter(forecast_years['ds'].iloc[-20], predictions_1_year, color='red', label='1 Year Prediction', s=50)
ax.scatter(forecast_years['ds'].iloc[-10], predictions_10_year, color='green', label='10 Years Prediction', s=50)
ax.scatter(forecast_years['ds'].iloc[-1], predictions_20_year, color='blue', label='20 Years Prediction', s=50)

ax.set_xlabel('Date')
ax.set_ylabel('Sunspots')
ax.set_title('Sunspot Predictions (Years)')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

fig2 = model.plot_components(forecast_years)
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

# Function to evaluate the model
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

year_df = year_df.rename(columns={'year': 'ds', 'total_sunspots': 'y'})

# Create and fit the Prophet model
model = Prophet()
model.fit(year_df)

# Create future dates for prediction
future = model.make_future_dataframe(periods=365, freq='D')

# Make predictions
forecast = model.predict(future)

# Extract predictions for specific future dates
predictions_100 = forecast.iloc[-100]['yhat']
predictions_200 = forecast.iloc[-200]['yhat']
predictions_365 = forecast.iloc[-365]['yhat']

# Forecasting with different parameters and evaluations

for growth in ['logistic', 'linear', 'flat']:
    for period in [365, 182, 91]: #Example periods
        for fourier_order in [5,10]: #Example Fourier Orders
          for n_changepoints in [10, 25]:
              for changepoint_prior_scale in [0.05, 0.5]:
                  model = Prophet(growth=growth, n_changepoints=n_changepoints, changepoint_prior_scale=changepoint_prior_scale)
                  model.add_seasonality(name='yearly', period=period, fourier_order=fourier_order)

                  if growth == 'logistic':
                    year_df['cap'] = year_df['y'].max() * 1.2
                  model.fit(year_df)

                  future = model.make_future_dataframe(periods=365, freq='D')

                  if growth == 'logistic':
                    future['cap'] = year_df['cap'].iloc[0]

                  forecast = model.predict(future)

                  #Evaluate the model
                  y_true = year_df['y'][-len(forecast['yhat']):] #Take the relevant part for comparision
                  y_pred = forecast['yhat'][-len(y_true):]
                  mae, mape, r2 = evaluate_model(y_true, y_pred)

                  print(f"Growth: {growth}, Period: {period}, Fourier Order: {fourier_order}, n_changepoints: {n_changepoints}, changepoint_prior_scale: {changepoint_prior_scale}")
                  print(f"MAE: {mae}, MAPE: {mape}, R^2: {r2}")

                  # Print predictions (example for 100 days)
                  print("Predictions (last 100 days):")
                  print(forecast[['ds', 'yhat']].tail(100))

                  fig1 = model.plot(forecast)
                  plt.title(f"Growth: {growth}, Period: {period}, Fourier Order: {fourier_order}")
                  plt.show()

                  fig2 = model.plot_components(forecast)
                  plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

# Function to evaluate the model
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

year_df = year_df.rename(columns={'year': 'ds', 'total_sunspots': 'y'})

future_months = model.make_future_dataframe(periods=9, freq='MS') #MS: Month Start

# Make predictions
forecast_months = model.predict(future_months)

# Extract predictions for 1, 6, and 9 months into the future
predictions_1_month = forecast_months.iloc[-9]['yhat'] #last 9 months
predictions_6_months = forecast_months.iloc[-4]['yhat']
predictions_9_months = forecast_months.iloc[-1]['yhat']

print(f"Sunspot prediction 1 month into the future: {predictions_1_month}")
print(f"Sunspot prediction 6 months into the future: {predictions_6_months}")
print(f"Sunspot prediction 9 months into the future: {predictions_9_months}")

# Forecasting with different parameters and evaluations

for growth in ['logistic', 'linear', 'flat']:
    for period in [365, 182, 91]: #Example periods
        for fourier_order in [5,10]: #Example Fourier Orders
          for n_changepoints in [10, 25]:
              for changepoint_prior_scale in [0.05, 0.5]:
                  model = Prophet(growth=growth, n_changepoints=n_changepoints, changepoint_prior_scale=changepoint_prior_scale)
                  model.add_seasonality(name='yearly', period=period, fourier_order=fourier_order)

                  if growth == 'logistic':
                    year_df['cap'] = year_df['y'].max() * 1.2
                  model.fit(year_df)

                  future = model.make_future_dataframe(periods=365, freq='D')

                  if growth == 'logistic':
                    future['cap'] = year_df['cap'].iloc[0]

                  forecast = model.predict(future)

                  #Evaluate the model
                  y_true = year_df['y'][-len(forecast['yhat']):] #Take the relevant part for comparision
                  y_pred = forecast['yhat'][-len(y_true):]
                  mae, mape, r2 = evaluate_model(y_true, y_pred)

                  print(f"Growth: {growth}, Period: {period}, Fourier Order: {fourier_order}, n_changepoints: {n_changepoints}, changepoint_prior_scale: {changepoint_prior_scale}")
                  print(f"MAE: {mae}, MAPE: {mape}, R^2: {r2}")

                  # Print predictions (example for 100 days)
                  print("Predictions (last 100 days):")
                  print(forecast[['ds', 'yhat']].tail(100))

                  fig1 = model.plot(forecast)
                  plt.title(f"Growth: {growth}, Period: {period}, Fourier Order: {fourier_order}")
                  plt.show()

                  fig2 = model.plot_components(forecast)
                  plt.show()

In [ ]:
# Function to evaluate the model
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

year_df = year_df.rename(columns={'year': 'ds', 'total_sunspots': 'y'})

future_years = model.make_future_dataframe(periods=20, freq='Y')
forecast_years = model.predict(future_years)

# Extract predictions
predictions_1_year = forecast_years.iloc[-20]['yhat']
predictions_10_year = forecast_years.iloc[-10]['yhat']
predictions_20_year = forecast_years.iloc[-1]['yhat']

print(f"Sunspot prediction 1 year into the future: {predictions_1_year}")
print(f"Sunspot prediction 10 years into the future: {predictions_10_year}")
print(f"Sunspot prediction 20 years into the future: {predictions_20_year}")

# Forecasting with different parameters and evaluations

for growth in ['logistic', 'linear', 'flat']:
    for period in [365, 182, 91]: #Example periods
        for fourier_order in [5,10]: #Example Fourier Orders
          for n_changepoints in [10, 25]:
              for changepoint_prior_scale in [0.05, 0.5]:
                  model = Prophet(growth=growth, n_changepoints=n_changepoints, changepoint_prior_scale=changepoint_prior_scale)
                  model.add_seasonality(name='yearly', period=period, fourier_order=fourier_order)

                  if growth == 'logistic':
                    year_df['cap'] = year_df['y'].max() * 1.2
                  model.fit(year_df)

                  future = model.make_future_dataframe(periods=365, freq='D')

                  if growth == 'logistic':
                    future['cap'] = year_df['cap'].iloc[0]

                  forecast = model.predict(future)

                  #Evaluate the model
                  y_true = year_df['y'][-len(forecast['yhat']):] #Take the relevant part for comparision
                  y_pred = forecast['yhat'][-len(y_true):]
                  mae, mape, r2 = evaluate_model(y_true, y_pred)

                  print(f"Growth: {growth}, Period: {period}, Fourier Order: {fourier_order}, n_changepoints: {n_changepoints}, changepoint_prior_scale: {changepoint_prior_scale}")
                  print(f"MAE: {mae}, MAPE: {mape}, R^2: {r2}")

                  # Print predictions (example for 100 days)
                  print("Predictions (last 100 days):")
                  print(forecast[['ds', 'yhat']].tail(100))

                  fig1 = model.plot(forecast)
                  plt.title(f"Growth: {growth}, Period: {period}, Fourier Order: {fourier_order}")
                  plt.show()

                  fig2 = model.plot_components(forecast)
                  plt.show()